# Predicting Match Outcomes

In [1]:
#SQL Imports
import mysql.connector
#Pandas imports
import pandas as pd
import datetime

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)
mycursor = mydb.cursor()

In [3]:
query = pd.read_sql_query('''
    SELECT * 
    FROM TeamMatchStats
''', mydb)

In [4]:
match_stat_df = pd.DataFrame(query)

In [5]:
match_stat_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,conversion_percentage,...,prop_1,prop_2,hooker,sr_1,sr_2,lock_1,int_1,int_2,int_3,int_4
0,1,1,14,1,1,34,5,4,3,0.8000,...,404,396,391,407,406,483,1626,402,397,219
1,2,1,1,14,0,12,2,1,1,0.5000,...,11,500,499,2,10,269,3,400,9,1624
2,3,2,8,6,1,19,3,3,0,1.0000,...,192,503,201,196,188,197,190,429,194,505
3,4,2,6,8,0,18,3,2,1,0.6667,...,510,509,153,511,155,512,516,309,513,327
4,5,3,10,4,1,20,4,1,1,0.2500,...,268,272,265,264,375,266,260,276,267,261


# Prediction Algorithm

Things to try:
1. Want to create scoring function (sklearn.metrics import f1_score, make_scorer, classification_report)

2. Random Forest
3. Neural Network (10-3-2)

Features to try:
- home field advantage
- form in last X matches
- Points differential last X matches
- Points scored last X matches
- Points allowed last X matches
- Running metre differential last X matches
- Tackle Percentages
- Factor in the strength of opposition


In [6]:
correlation_df = match_stat_df
col = list(match_stat_df.columns)[35:52]
correlation_df = match_stat_df.drop(col, axis=1)
correlation_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,conversion_percentage,...,ineffective_tackles,tackle_percentage,kicks,kicking_metres,metres_per_kick,errors,handling_errors,penalties,sin_bins,send_offs
0,1,1,14,1,1,34,5,4,3,0.8000,...,14,0.8881,17,478,28.1176,10,9,7,0,0
1,2,1,1,14,0,12,2,1,1,0.5000,...,15,0.8715,14,382,27.2857,13,11,9,0,0
2,3,2,8,6,1,19,3,3,0,1.0000,...,27,0.8465,21,538,25.6190,6,6,9,0,0
3,4,2,6,8,0,18,3,2,1,0.6667,...,5,0.8883,13,284,21.8462,16,10,11,0,0
4,5,3,10,4,1,20,4,1,1,0.2500,...,19,0.8774,13,252,19.3846,15,10,6,0,0


In [7]:
correlation_df = correlation_df.drop(columns=['line_engaged_runs'])

In [8]:
import seaborn as sns
corr = correlation_df.corr()
corr['is_winner']
stats = corr[(corr['is_winner'] > .2) | (corr['is_winner'] < -.2)]
stats['is_winner']
#corr.style.background_gradient(cmap='coolwarm')

is_winner              1.000000
points                 0.616395
tries                  0.563085
conversions            0.495853
penalty_goals          0.331492
field_goals            0.213446
runs                   0.291058
run_metres             0.387814
kick_return_metres     0.257286
post_contact_metres    0.253308
line_breaks            0.372330
tackle_breaks          0.291125
hit_ups                0.295609
tackles               -0.330251
missed_tackles        -0.290167
kicks                  0.236040
kicking_metres         0.247338
errors                -0.236594
handling_errors       -0.203090
Name: is_winner, dtype: float64

Columns to take into consideration
From highly correlated stats:

- points (allowed)
- tries (allowed)
- penalty_goals
- run_metres (allowed)
- runs
- runs_per_metre (allowed)
- line_breaks (allowed)
- tackle_breaks
- tackles (allowed)
- missed_tackles
- kicking_metres
- errors

In [28]:
stat_columns = ['id', 'match_id', 'team_id', 'opponent_id', 'is_winner', 'points', 'tries', 'penalty_goals',
                     'run_metres', 'runs', 'line_breaks', 'tackle_breaks', 'tackles', 'missed_tackles',
                     'kicking_metres', 'errors']
results_df = match_stat_df[stat_columns]
results_df['metres_per_run'] = results_df['run_metres'] / results_df['runs']
results_df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,id,match_id,team_id,opponent_id,is_winner,points,tries,penalty_goals,run_metres,runs,line_breaks,tackle_breaks,tackles,missed_tackles,kicking_metres,errors,metres_per_run
0,1,1,14,1,1,34,5,3,1618,168,5,35,238,16,478,10,9.630952
1,2,1,1,14,0,12,2,1,1076,130,2,16,339,35,382,13,8.276923
2,3,2,8,6,1,19,3,0,1425,192,3,41,342,35,538,6,7.421875
3,4,2,6,8,0,18,3,1,1648,178,4,35,366,41,284,16,9.258427
4,5,3,10,4,1,20,4,1,1348,165,3,37,272,19,252,15,8.169697


columns to create

- points (allowed)
- tries (allowed)
- penalty_goals
- run_metres (allowed)
- runs
- runs_per_metre (allowed)
- line_breaks (allowed)
- tackle_breaks
- tackles (allowed)
- missed_tackles
- kicking_metres
- errors

- tackle_percentage
- metres_per_run

- point_differential
- run_metre differential
- 

- home field advantage
- form in last X matches
- Points differential last X matches
- Points scored last X matches
- Points allowed last X matches
- Running metre differential last X matches
- line_breaks last X matches
- errors last X matches
- Tackle Percentages
- Strength of opposition metric

In [29]:
columns = [*results_df.columns.tolist(),'opp_points', 'opp_tries', 'opp_run_metres',
           'opp_metres_per_run', 'opp_line_breaks', 'opp_tackles', 'point_diff', 'run_metre_diff']
results_df = results_df.reindex(columns=columns, fill_value=0)
#print(predictions_df)

for match in list(results_df['match_id'].unique()):
    match_df = results_df[results_df['match_id'] == match]
    #print(match_df)
    for team in list(match_df['team_id']):
        team_data = match_df[match_df['team_id'] == team]
        opp_data = match_df[match_df['team_id'] != team]
        
        opp_points = opp_data['points'].iloc[0]
        opp_tries = opp_data['tries'].iloc[0]
        opp_run_metres = opp_data['run_metres'].iloc[0]
        opp_metres_per_run = opp_data['metres_per_run'].iloc[0]
        opp_line_breaks = opp_data['line_breaks'].iloc[0]
        opp_tackles = opp_data['tackles'].iloc[0]
        
        point_diff = team_data['points'].iloc[0] - opp_points
        run_metre_diff = team_data['run_metres'].iloc[0] - opp_run_metres
        
        results_df.loc[team_data.index, 'opp_points'] = opp_points
        results_df.loc[team_data.index, 'opp_tries'] = opp_tries
        results_df.loc[team_data.index, 'opp_run_metres'] = opp_run_metres
        results_df.loc[team_data.index, 'opp_metres_per_run'] = opp_metres_per_run
        results_df.loc[team_data.index, 'opp_line_breaks'] = opp_line_breaks
        results_df.loc[team_data.index, 'opp_tackles'] = opp_tackles
        results_df.loc[team_data.index, 'point_diff'] = point_diff
        results_df.loc[team_data.index, 'run_metre_diff'] = run_metre_diff
          
results_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,penalty_goals,run_metres,runs,...,errors,metres_per_run,opp_points,opp_tries,opp_run_metres,opp_metres_per_run,opp_line_breaks,opp_tackles,point_diff,run_metre_diff
0,1,1,14,1,1,34,5,3,1618,168,...,10,9.630952,12,2,1076,8.276923,2,339,22,542
1,2,1,1,14,0,12,2,1,1076,130,...,13,8.276923,34,5,1618,9.630952,5,238,-22,-542
2,3,2,8,6,1,19,3,0,1425,192,...,6,7.421875,18,3,1648,9.258427,4,366,1,-223
3,4,2,6,8,0,18,3,1,1648,178,...,16,9.258427,19,3,1425,7.421875,3,342,-1,223
4,5,3,10,4,1,20,4,1,1348,165,...,15,8.169697,14,2,1144,8.537313,1,323,6,204


In [30]:
def compute_past_x_match_avg(x, stat, data):
    x_match_avg = []
    for index in range(len(data)):
        if index <= (x - 1):
            start = 0
        else:
            start = (index - x)
        past_x_matches = data.iloc[start : index]
        avg = past_x_matches[stat].mean()
        x_match_avg.append(avg)
    return pd.Series(x_match_avg).values

In [31]:
columns = results_df.columns.tolist()
stats = ['is_winner', 'points', 'tries', 'penalty_goals', 'run_metres', 'runs', 'line_breaks',
         'tackle_breaks', 'tackles', 'missed_tackles', 'kicking_metres', 'errors']

for stat in stats:
    for x in [4, 8]:
        columns.append('avg_' + stat + '_past_' + str(x) + '_matches')
results_df = results_df.reindex(columns=columns, fill_value=0)

for team in list(results_df['team_id'].unique()):
    team_df = results_df[results_df['team_id'] == team]
    team_df = team_df.reset_index()
    for stat in stats:
        for x in [4, 8]:
            new_column = 'avg_' + stat + '_past_' + str(x) + '_matches'
            team_df[new_column] = compute_past_x_match_avg(x, stat, team_df)
    team_df = team_df.set_index('index')
    results_df.loc[results_df['team_id'] == team] = team_df
    #print(predictions_df[predictions_df['team_id'] == team])
print(results_df[results_df['team_id'] == 12])

      id  match_id  team_id  opponent_id  is_winner  points  tries  \
12    13         7       12           11          1      24      3   
24    25        13       12           13          1      18      3   
35    36        18       12            3          0      18      3   
49    50        25       12           10          1      33      4   
79    80        40       12           11          1      12      1   
92    93        47       12            5          1      35      6   
111  112        56       12            4          0      22      4   
120  121        61       12            3          1      22      3   
132  133        67       12           10          0      20      4   
147  148        74       12            8          1      29      4   
160  161        81       12           16          1      16      2   
186  187        94       12           14          1      28      4   
201  202       101       12            2          1      23      3   
221  222       111  

In [32]:
results_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,penalty_goals,run_metres,runs,...,avg_tackle_breaks_past_4_matches,avg_tackle_breaks_past_8_matches,avg_tackles_past_4_matches,avg_tackles_past_8_matches,avg_missed_tackles_past_4_matches,avg_missed_tackles_past_8_matches,avg_kicking_metres_past_4_matches,avg_kicking_metres_past_8_matches,avg_errors_past_4_matches,avg_errors_past_8_matches
0,1,1,14,1,1,34,5,3,1618,168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,14,0,12,2,1,1076,130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2,8,6,1,19,3,0,1425,192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2,6,8,0,18,3,1,1648,178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,3,10,4,1,20,4,1,1348,165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
query = pd.read_sql_query('''
    SELECT * 
    FROM Matches
''', mydb)
predictions_df = pd.DataFrame(query)
predictions_df.head()

In [ ]:
predictions_df = predictions_df[['id', 'round', 'home_team_id', 'home_score', 'away_team_id', 'away_score', 'winner', 'home_odds', 'away_odds']]

In [ ]:
columns = list(results_df.drop(columns=['id', 'match_id', 'team_id', 'opponent_id', 'is_winner', 'points',
                                    'tries', 'penalty_goals', 'run_metres', 'runs', 'line_breaks',
                                    'tackle_breaks', 'tackles', 'missed_tackles', 'kicking_metres',
                                    'errors', 'metres_per_run', 'opp_points', 'opp_tries', 'opp_run_metres',
                                    'opp_metres_per_run', 'opp_line_breaks', 'opp_tackles', 'point_diff',
                                    'run_metre_diff']))

columns = predictions_df.columns.tolist()
for team in ['h', 'a']:
    for column in columns:
        columns.append(team + '_' + column)
print(columns)
#c = [*predictions_df.columns.tolist().append(new_columns)

#columns = [*results_df.columns.tolist(),'opp_points', 'opp_tries', 'opp_run_metres',
           'opp_metres_per_run', 'opp_line_breaks', 'opp_tackles', 'point_diff', 'run_metre_diff']
predictions_df = predictions_df.reindex(columns=columns, fill_value=0)

for match in predictions_df.iterrows():
    match = match[1]
    home_team = match['home_team_id']
    away_team = match['away_team_id']
    match_result_data = results_df.loc[results_df['match_id'] == match['id']]
    home_data = match_result_data[match_result_data['team_id'] == home_team]
    away_data = match_result_data[match_result_data['team_id'] == away_team]
    for team in ['h', 'a']:
        if team == 'h':
            df = home_data
        else:
            df = away_data
        df = df.drop(columns=['id', 'match_id', 'team_id', 'opponent_id', 'is_winner', 'points',
                                    'tries', 'penalty_goals', 'run_metres', 'runs', 'line_breaks',
                                    'tackle_breaks', 'tackles', 'missed_tackles', 'kicking_metres',
                                    'errors', 'metres_per_run', 'opp_points', 'opp_tries', 'opp_run_metres',
                                    'opp_metres_per_run', 'opp_line_breaks', 'opp_tackles', 'point_diff',
                                     'run_metre_diff'])
        for column in list(df.columns):
            col = team + '_' + column
            print(col)
            print(column)
            match[col] = team[column]

#predictions_df = predictions_df.apply(lambda x: add_data_to_prediction_df(x))
predictions_df.head()

In [ ]:
mycursor.close()